<a href="https://colab.research.google.com/github/ababyyds/3dgs/blob/main/gaussian_splatting_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn



# !wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
# !unzip GaussianViewTest.zip
# !python render.py -m /content/gaussian-splatting/GaussianViewTest/model
# !ffmpeg -framerate 3 -i /content/gaussian-splatting/GaussianViewTest/model/train/ours_30000/renders/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /content/renders.mp4
# !ffmpeg -framerate 3 -i /content/gaussian-splatting/GaussianViewTest/model/train/ours_30000/gt/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /content/gt.mp4 -y

/content
Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 603, done.
remote: Total 603 (delta 0), reused 0 (delta 0), pack-reused 603 (from 1)
Receiving objects: 100% (603/603), 2.09 MiB | 19.65 MiB/s, done.
Resolving deltas: 100% (347/347), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3270, done.        
remote: Counting objects: 100% (299/299), done.        
remote: Compressing objects: 100% (164/164), done.        
remote: Total 3270 (delta 157), reused 259 (delta 135), pack-reused 2971 (from 1)        
Receiving objects:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [3]:
!python train.py -s /content/gaussian-splatting/dataset/dog -m /content/gaussian-splatting/dataset/dog/output

2024-11-24 13:11:38.423243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 13:11:38.443009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 13:11:38.448956: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 13:11:38.463093: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-24 13:11:39.467923: W tensorflow/comp

In [4]:
!python render.py -m /content/gaussian-splatting/dataset/dog/output

Looking for config file in /content/gaussian-splatting/dataset/dog/output/cfg_args
Config file found: /content/gaussian-splatting/dataset/dog/output/cfg_args
Rendering /content/gaussian-splatting/dataset/dog/output
Loading trained model at iteration 30000 [24/11 14:36:57]
Reading camera 72/72 [24/11 14:36:57]
Loading Training Cameras [24/11 14:36:57]
[ INFO ] Encountered quite large input images (>1.6K pixels width), rescaling to 1.6K.
 If this is not desired, please explicitly specify '--resolution/-r' as 1 [24/11 14:36:57]
Loading Test Cameras [24/11 14:37:12]
Rendering progress: 100% 72/72 [01:50<00:00,  1.53s/it]
Rendering progress: 0it [00:00, ?it/s]


In [7]:
pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.3 MB/s eta 0:00:00


In [9]:
import os
import skimage.io
import skimage.metrics
import lpips
import torch
from torchvision import transforms

# 定义渲染图像路径和真实图像路径
renders_dir = "/content/gaussian-splatting/dataset/dog/output/train/ours_30000/renders"
gt_dir = "/content/gaussian-splatting/dataset/dog/output/train/ours_30000/gt"

# 获取所有图像文件名
image_filenames = [f for f in os.listdir(renders_dir) if f.endswith(".png")]

# 初始化指标列表
psnr_values = []
ssim_values = []
lpips_values = []
loss_fn_alex = lpips.LPIPS(net='alex') # 最好使用alex网络，vgg速度会比较慢
to_tensor = transforms.ToTensor()
# 遍历所有图像
for filename in image_filenames:
    # 加载渲染图像和真实图像
    img_pred = skimage.io.imread(os.path.join(renders_dir, filename))
    img_gt = skimage.io.imread(os.path.join(gt_dir, filename))

    # 计算 PSNR 和 SSIM
    psnr = skimage.metrics.peak_signal_noise_ratio(img_gt, img_pred)
    try:
        ssim = skimage.metrics.structural_similarity(img_gt, img_pred, multichannel=True, win_size=3)
    except ValueError:
        # 如果仍然报错，则跳过此图像的 SSIM 计算
        print(f"Skipping SSIM calculation for {filename} due to size limitations.")
        ssim = float('nan')  # 使用 NaN 表示 SSIM 值不可用
    img_pred_tensor = to_tensor(img_pred).unsqueeze(0)  # 添加批次维度
    img_gt_tensor = to_tensor(img_gt).unsqueeze(0)
    lpips_value = loss_fn_alex(img_pred_tensor, img_gt_tensor)
    # 将指标值添加到列表中
    psnr_values.append(psnr)
    ssim_values.append(ssim)
    lpips_values.append(lpips_value)

# 计算平均 PSNR 和 SSIM
avg_psnr = sum(psnr_values) / len(psnr_values)
avg_ssim = sum(ssim_values) / len(ssim_values)
avg_lpips = sum(lpips_values) / len(lpips_values)
# 打印平均指标结果
print(f"Average PSNR: {avg_psnr}")
print(f"Average SSIM: {avg_ssim}")
print(f"Average LPIPS: {avg_lpips}")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'), st

Average PSNR: 41.38657122400864
Average SSIM: 0.9954849413838299
Average LPIPS: tensor([[[[0.0146]]]], grad_fn=<DivBackward0>)


In [1]:
pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7

In [ ]:
import numpy as np
import open3d as o3d
import cv2
import struct

def read_cameras_binary(path):
    cameras = {}
    with open(path, "rb") as f:
        num_cameras = struct.unpack("<Q", f.read(8))[0]
        for _ in range(num_cameras):
            camera_id = struct.unpack("<I", f.read(4))[0]
            model = struct.unpack("<I", f.read(4))[0]
            width = struct.unpack("<Q", f.read(8))[0]
            height = struct.unpack("<Q", f.read(8))[0]
            params = struct.unpack("<" + "d" * 4, f.read(8 * 4))
            cameras[camera_id] = {
                "model": model,
                "width": width,
                "height": height,
                "params": params
            }
    return cameras

def read_images_binary(path):
    with open(path, "rb") as f:
        num_images = struct.unpack("<Q", f.read(8))[0]
        images = {}
        for _ in range(num_images):
            image_id = struct.unpack("<I", f.read(4))[0]
            qvec = struct.unpack("<" + "d" * 4, f.read(8 * 4))  # 读取四元数
            tvec = struct.unpack("<" + "d" * 3, f.read(8 * 3))  # 读取平移向量
            camera_id = struct.unpack("<I", f.read(4))[0]
            name = ""
            while True:
                char = f.read(1).decode("utf-8")
                if char == '\x00':
                    break
                name += char
            num_points2D = struct.unpack("<Q", f.read(8))[0]
            points2D = []
            for _ in range(num_points2D):
                x = struct.unpack("<d", f.read(8))[0]
                y = struct.unpack("<d", f.read(8))[0]
                point3D_id = struct.unpack("<q", f.read(8))[0]
                points2D.append((x, y, point3D_id))
            images[image_id] = {
                "qvec": qvec,
                "tvec": tvec,
                "camera_id": camera_id,
                "name": name,
                "points2D": points2D
            }
    return images

def quaternion_to_rotation_matrix(qvec):
    q0, q1, q2, q3 = qvec
    R = np.array([
        [1 - 2 * (q2**2 + q3**2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
        [2 * (q1 * q2 + q0 * q3), 1 - 2 * (q1**2 + q3**2), 2 * (q2 * q3 - q0 * q1)],
        [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), 1 - 2 * (q1**2 + q2**2)]
    ])
    return R

def load_colmap_data(cameras_bin_path, images_bin_path, ply_path):
    cameras = read_cameras_binary(cameras_bin_path)
    images = read_images_binary(images_bin_path)
    pcd = o3d.io.read_point_cloud(ply_path)
    return cameras, images, pcd

def render_image(cameras, images, pcd, image_id):
    image_param = images[image_id]
    camera_param = cameras[image_param['camera_id']]

    # 设置相机内参
    fx, fy, cx, cy = camera_param['params']
    intrinsic = o3d.camera.PinholeCameraIntrinsic()
    intrinsic.set_intrinsics(camera_param['width'], camera_param['height'], fx, fy, cx, cy)

    # 设置相机外参
    R = quaternion_to_rotation_matrix(image_param['qvec'])
    t = np.array(image_param['tvec']).reshape(3, 1)
    extrinsic = np.eye(4)
    extrinsic[:3, :3] = R
    extrinsic[:3, 3] = t.flatten()

    # 创建渲染器
    render = o3d.visualization.rendering.OffscreenRenderer(camera_param['width'], camera_param['height'])
    render.scene.add_geometry("pcd", pcd, o3d.visualization.rendering.MaterialRecord())
    render.setup_camera(intrinsic, extrinsic)
    img = render.render_to_image()

    # 转换为 OpenCV 格式
    img = np.asarray(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

if __name__ == "__main__":
    # 设置路径
    cameras_bin_path = "/content/cameras.bin"
    images_bin_path = "/content/images.bin"
    ply_path = "/content/fused.ply"

    # 加载 COLMAP 数据
    cameras, images, pcd = load_colmap_data(cameras_bin_path, images_bin_path, ply_path)

    # 渲染图像
    image_id = 1  # 指定要渲染的图像 ID
    rendered_image = render_image(cameras, images, pcd, image_id)

    # 保存渲染图像
    cv2.imwrite("rendered_image.png", rendered_image)